In [1]:
import numpy as np
import pandas as pd
import os 
from model import MyModel
import tensorflow as tf
from matplotlib.pyplot import cm
import matplotlib.pyplot as plt
import numpy as np
import h5py
from data.plot3D import plot_voxelgrid, plot_points
from  data.voxelgrid import VoxelGrid